# Build custom container for Vertex training

In [1]:
!pwd

/home/jupyter/tf_vertex_agents/04-perarm-features-bandit


## Load env config

* use the prefix from `00-env-setup`

In [2]:
PREFIX = 'mabv1'

In [3]:
# staging GCS
GCP_PROJECTS             = !gcloud config get-value project
PROJECT_ID               = GCP_PROJECTS[0]

# GCS bucket and paths
BUCKET_NAME              = f'{PREFIX}-{PROJECT_ID}-bucket'
BUCKET_URI               = f'gs://{BUCKET_NAME}'

config = !gsutil cat {BUCKET_URI}/config/notebook_env.py
print(config.n)
exec(config.n)


PROJECT_ID               = "hybrid-vertex"
PROJECT_NUM              = "934903580331"
LOCATION                 = "us-central1"

REGION                   = "us-central1"
BQ_LOCATION              = "US"
VPC_NETWORK_NAME         = "ucaip-haystack-vpc-network"

VERTEX_SA                = "934903580331-compute@developer.gserviceaccount.com"

PREFIX                   = "mabv1"
VERSION                  = "v1"

BUCKET_NAME              = "mabv1-hybrid-vertex-bucket"
DATA_GCS_PREFIX          = "data"
DATA_PATH                = "gs://mabv1-hybrid-vertex-bucket/data"
BUCKET_URI               = "gs://mabv1-hybrid-vertex-bucket"

VPC_NETWORK_FULL         = "projects/934903580331/global/networks/ucaip-haystack-vpc-network"

BIGQUERY_DATASET_ID      = "hybrid-vertex.movielens_dataset_mabv1"
BIGQUERY_TABLE_ID        = "hybrid-vertex.movielens_dataset_mabv1.training_dataset"

REPO_DOCKER_PATH_PREFIX  = "src"
RL_SUB_DIR               = "per_arm_rl"



In [4]:
! gsutil ls $BUCKET_URI

gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-13-22:02:47.458/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-14-12:09:06.643/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-14-12:46:07.509/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-14-13:48:08.938/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-17-18:35:00.675/
gs://mabv1-hybrid-vertex-bucket/aiplatform-custom-training-2023-07-17-18:50:42.146/
gs://mabv1-hybrid-vertex-bucket/archived/
gs://mabv1-hybrid-vertex-bucket/config/
gs://mabv1-hybrid-vertex-bucket/custom-neural-bandits-a100/
gs://mabv1-hybrid-vertex-bucket/custom-neural-bandits-v1/
gs://mabv1-hybrid-vertex-bucket/data/
gs://mabv1-hybrid-vertex-bucket/linear-bandit-v1/
gs://mabv1-hybrid-vertex-bucket/neural-bandit-v2/
gs://mabv1-hybrid-vertex-bucket/neural-linear-bandits-v1/
gs://mabv1-hybrid-vertex-bucket/perarm-local-test/
gs://mabv1-hybrid-vertex-bucket/scale-perarm-hpt-

## imports

In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Build Image

## Container Image Variables

In [6]:
DOCKERNAME                = f'Dockerfile_perarm_feats'

REPOSITORY                = f'rl-movielens-{PREFIX}'
IMAGE_NAME                = f'train-perarm-feats-{VERSION}'

REMOTE_IMAGE_NAME         = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE_NAME}"

print(f"DOCKERNAME        = {DOCKERNAME}")
print(f"REPOSITORY        = {REPOSITORY}")
print(f"IMAGE_NAME        = {IMAGE_NAME}")
print(f"REMOTE_IMAGE_NAME = {REMOTE_IMAGE_NAME}")

DOCKERNAME        = Dockerfile_perarm_feats
REPOSITORY        = rl-movielens-mabv1
IMAGE_NAME        = train-perarm-feats-v1
REMOTE_IMAGE_NAME = us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-mabv1/train-perarm-feats-v1


## Create Artifact Repository

If you don't have an existing artifact repository, create one using the gcloud command below

In [8]:
! gcloud artifacts repositories create $REPOSITORY --repository-format=docker --location=$LOCATION

Create request issued for: [rl-movielens-mabv1]
Waiting for operation [projects/hybrid-vertex/locations/us-central1/operations/
dd1c81a6-d64a-493a-87cc-ab9bb68ad432] to complete...done.                      
Created repository [rl-movielens-mabv1].


## Local Docker build

Provide a name for your dockerfile and make sure you are authenticated

In [7]:
! gcloud auth configure-docker $REGION-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


### Create Dockerfile

In [9]:
import os

root_path = '..'
os.chdir(root_path)
os.getcwd()

'/home/jupyter/tf_vertex_agents'

In [10]:
%%writefile {DOCKERNAME}

FROM python:3.10

ENV PYTHONUNBUFFERED True

ENV APP_HOME /workspace

WORKDIR $APP_HOME

COPY requirements.txt $APP_HOME/requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir -r $APP_HOME/requirements.txt

# ADD src $APP_HOME/src
# RUN ls src
COPY src/perarm_features $APP_HOME/src/perarm_features
COPY src/per_arm_rl $APP_HOME/src/per_arm_rl

RUN export PYTHONPATH=${PYTHONPATH}:${APP_HOME}/

# Sets up the entry point to invoke the task.
ENTRYPOINT ["python3", "-m", "src.perarm_features.task"]

Overwriting Dockerfile_perarm_feats


### Build Image Locally

In [13]:
!docker build -t $REMOTE_IMAGE_NAME -f $DOCKERNAME .

### Push container to Registry

In [12]:
# ### push the container to registry
!docker push $REMOTE_IMAGE_NAME

Using default tag: latest
The push refers to repository [us-central1-docker.pkg.dev/hybrid-vertex/rl-movielens-mabv1/train-perarm-feats-v1]

cf9f0773: Preparing 
f7384de3: Preparing 
027a3cc3: Preparing 
0cfcba49: Preparing 
5a94bce1: Preparing 
c433112d: Preparing 
98117129: Preparing 
ef3df566: Preparing 
1ac8c9e8: Preparing 
d4dd95f0: Preparing 
221bdf24: Preparing 
f477cd5d: Preparing 
f9991a11: Preparing 
27a3cc3: Pushed   1.975GB/1.948GBlatest: digest: sha256:de1007996cc1c4052781a5d336ada9dc97d71d115a9a546e7488a7dc6e6a5e9c size: 3264
